In [ ]:
import json
import requests
from requests.auth import HTTPBasicAuth

# PARAMETERS

These parameters won't change during a Central Model Registry configuration.

In [ ]:
sp_tenant_id = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXX" #Tenant ID of the Service Principal

In [ ]:
db_admin_username = "daniel.perez@databricks.com" #Admin for Central Model Registry
db_admin_token = "dapiXXXXXXXXXXXXXXXXXXXXXXXXXX" #Central Model Registry
db_url = "adb-XXXXXXXXXXXXX.1.azuredatabricks.net" #Central Model Registry

These parameters will change for every Satellite Workspace during a Central Model Registry configuration.

In [ ]:
sp_display_name = "dp-bci-test" #Name of the Service Principal
sp_application_id = "XXXXXXXX-XXXXX-XXXXX-XXXXX-XXXXXXXXX" #Application ID for the Service Principal
sp_client_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXX" #Client secret for the service principal

In [ ]:
pat_token_comment = f"Token for ML {sp_display_name}" # Comment for Databricks
pat_token_lifetime_seconds = -1 # Perpetual token: -1

# MAIN

## ADD SERVICE PRINCIPAL TO CENTRAL WORKSPACE

In [ ]:
add_sp_json = {
  "schemas": [ "urn:ietf:params:scim:schemas:core:2.0:ServicePrincipal" ],
  "applicationId": sp_application_id,
  "displayName": sp_display_name
}

response = requests.post(f"https://{db_url}/api/2.0/preview/scim/v2/ServicePrincipals",
                        headers = {'Content-type': 'application/scim+json'},
                        auth=HTTPBasicAuth(db_admin_username, db_admin_token),
                        data=json.dumps(add_sp_json))

In [ ]:
response.content

## GRANT SERVICE PRINCIPAL PAT TOKEN PERMISSIONS ON CENTRALIZED WORKSPACE

In [ ]:
url = f"https://{db_url}/api/2.0/permissions/authorization/tokens"

payload = {
  "access_control_list": [
    {
      "user_name": sp_application_id,
      "permission_level": "CAN_USE"
    }
  ]
}

response = requests.patch(url,
                          headers = {'Content-type': 'application/json'},
                          auth=HTTPBasicAuth(db_admin_username, db_admin_token),
                          data=json.dumps(payload))

In [ ]:
response.content

## CREATE PAT TOKEN

### Get Bearer token for API

In [ ]:
url = f"https://login.microsoftonline.com/{sp_tenant_id}/oauth2/token"
payload=f'grant_type=client_credentials&client_id={sp_application_id}&client_secret={sp_client_secret}&resource=2ff814a6-3304-4ab8-85cb-cd0e6f879c1d'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = response.json().get("access_token")

In [ ]:
response.content

### Get PAT Token

In [ ]:
token_json = {
    "comment": pat_token_comment,
    "lifetime_seconds": pat_token_lifetime_seconds
}

response = requests.post(
    f"https://{db_url}/api/2.0/token/create",
    headers={"Authorization": f"Bearer {access_token}"},
    data=json.dumps(token_json)
)

In [ ]:
response.content

In [ ]:
sp_pat_token = response.json().get("token_value")

# vvvNOTE DOWN THIS VALUEvvv

In [ ]:
sp_display_name, sp_pat_token

# ^^^NOTE DOWN THIS VALUE^^^